# Data Preparation - Calculate movement rates - single ticker

Within this file, we want to calculate movement rates which will be used as independend variables instead of using just open or close value. The reason is that a movement rate ist better to understand and based on this it is easier to find a good decision.

## Content

 1. Import dependencies
 2. Load dataframe
     1. Define our series to supervised function
 3. Prepare dataframe
     1. Prepare dataframe for stockprice movement
     2. Calculate the increase of the stockprice within a difference of 90 days in percent
     3. Prepare dataframe for volume movement
     4. Calculate the increase of the volume within a difference of 90 days in percent
     5. Prepare dataframe for 5-day moving average
     6. Prepare dataframe for Exponential moving average
 4. Merge all dataframes
 5. Add trained label
 6. Save dataframe as .csv file
 
 

<hr>

## 1. Import dependencies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import Series
from sklearn import preprocessing
from  pandas  import  DataFrame 
from  pandas  import  concat 

%matplotlib inline

## 2. Load dataframe

#### Load dataframe from csv file

In [2]:
df_ticker = pd.read_csv("data/stocks/AAPL.csv")
df_ticker.columns = ['Date','Open','Close','Low','High','Volume']

df_ticker.head()

,Date,Open,Close,Low,High,Volume
0,2008-01-02,28.467142,27.834286,27.507143,28.608572,269794700.0
1,2008-01-03,27.915714,27.847143,27.527143,28.198572,210516600.0
2,2008-01-04,27.350000,25.721428,25.555714,27.571428,363958000.0
3,2008-01-07,25.892857,25.377142,24.318571,26.228571,518048300.0
4,2008-01-08,25.734285,24.464285,24.400000,26.065714,380954000.0


## A. Define function for convert our series to supervised learning using DataFrame.shift

**Time Series vs. Supervised learning**

A time series is a sequence of numbers that are ordered by a time index. This can be thought of as a list or column of ordered values.

A supervised learning problem is comprised of input patterns (X) and output patterns (y), such that an algorithm can learn how to predict the output patterns from the input patterns.


A key function to help transform time series data into a supervised learning problem is the Pandas shift() function.

Given a DataFrame, the shift() function can be used to create copies of columns that are pushed forward (rows of NaN values added to the front) or pulled back (rows of NaN values added to the end).

This is the behavior required to create columns of lag observations as well as columns of forecast observations for a time series dataset in a supervised learning format.

More details and the text sourece are found under following link: [Link](https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/)

In [3]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

# 3. Prepare dataframe and calculate movement rates

### A. Prepare dataframe for stockprice movement

#### Calculating mid prices (mean between High and Low) + Drop unused Columns in Dataframe

In [4]:
df_ticker['Mid_prices'] = (df_ticker.Low + df_ticker.High) / 2
df_for_midprices = df_ticker.copy(deep=True)
df_midprices = df_for_midprices.drop(['Date','Open','High','Low','Close','Volume'], axis=1)

#Normalize data using MinMaxScaler
scaler = preprocessing.MinMaxScaler()
df_midprices_values = scaler.fit_transform(df_midprices)
df_midprices = pd.DataFrame(df_midprices_values, columns=['Mid_prices'])
df_midprices.head()

,Mid_prices
0,0.075460
1,0.074575
2,0.068674
3,0.062815
4,0.062630


#### Convert Series to Supervised Learning Dataframe

In [5]:
# Using the Pandas Shift function to shift our dataframe + 90 days (function needs 91)
midprices = df_midprices.values
reframed_midprices = series_to_supervised(midprices, 0, 91)
reframed_midprices.head()

,var1(t),var1(t+1),var1(t+2),var1(t+3),var1(t+4),var1(t+5),var1(t+6),var1(t+7),var1(t+8),var1(t+9),...,var1(t+81),var1(t+82),var1(t+83),var1(t+84),var1(t+85),var1(t+86),var1(t+87),var1(t+88),var1(t+89),var1(t+90)
0,0.075460,0.074575,0.068674,0.062815,0.062630,0.060859,0.063652,0.060875,0.063061,0.059587,...,0.060246,0.062520,0.063149,0.064969,0.066880,0.067833,0.067652,0.067921,0.066640,0.068619
1,0.074575,0.068674,0.062815,0.062630,0.060859,0.063652,0.060875,0.063061,0.059587,0.053693,...,0.062520,0.063149,0.064969,0.066880,0.067833,0.067652,0.067921,0.066640,0.068619,0.071081
2,0.068674,0.062815,0.062630,0.060859,0.063652,0.060875,0.063061,0.059587,0.053693,0.053066,...,0.063149,0.064969,0.066880,0.067833,0.067652,0.067921,0.066640,0.068619,0.071081,0.070594
3,0.062815,0.062630,0.060859,0.063652,0.060875,0.063061,0.059587,0.053693,0.053066,0.053579,...,0.064969,0.066880,0.067833,0.067652,0.067921,0.066640,0.068619,0.071081,0.070594,0.069391
4,0.062630,0.060859,0.063652,0.060875,0.063061,0.059587,0.053693,0.053066,0.053579,0.047292,...,0.066880,0.067833,0.067652,0.067921,0.066640,0.068619,0.071081,0.070594,0.069391,0.070429


### B. Calculate the increase of the stockprice within a difference of 90 days in percent

In [6]:
entwicklungsrate_midprices_90 = (1/reframed_midprices['var1(t)']*reframed_midprices['var1(t+90)'])-1

##### Combine the developmentrate with the date

In [7]:
df_for_midprices['Entwicklungsrate Preis t+90'] = entwicklungsrate_midprices_90
df_for_midprices = df_for_midprices.drop(['Open','High','Low','Close','Volume','Mid_prices'], axis=1)
df_for_midprices.head()

,Date,Entwicklungsrate Preis t+90
0,2008-01-02,-0.090667
1,2008-01-03,-0.046851
2,2008-01-04,0.027965
3,2008-01-07,0.104684
4,2008-01-08,0.124521


### C. Prepare dataframe for volume movement

In [8]:
df_for_volumes = df_ticker.copy(deep=True)
df_volumes = df_for_volumes.drop(['Date','Open','High','Low','Close','Mid_prices'], axis=1)
#Normalize data
scaler = preprocessing.MinMaxScaler()
df_volumes_values = scaler.fit_transform(df_midprices)
df_volumes = pd.DataFrame(df_volumes_values, columns=['Volume'])
df_volumes.head()

,Volume
0,0.075460
1,0.074575
2,0.068674
3,0.062815
4,0.062630


#### Convert Series to Supervised Learning Dataframe

In [9]:
volumes = df_volumes.values
reframed_volumes = series_to_supervised(volumes, 0, 91)
reframed_volumes.head()

,var1(t),var1(t+1),var1(t+2),var1(t+3),var1(t+4),var1(t+5),var1(t+6),var1(t+7),var1(t+8),var1(t+9),...,var1(t+81),var1(t+82),var1(t+83),var1(t+84),var1(t+85),var1(t+86),var1(t+87),var1(t+88),var1(t+89),var1(t+90)
0,0.075460,0.074575,0.068674,0.062815,0.062630,0.060859,0.063652,0.060875,0.063061,0.059587,...,0.060246,0.062520,0.063149,0.064969,0.066880,0.067833,0.067652,0.067921,0.066640,0.068619
1,0.074575,0.068674,0.062815,0.062630,0.060859,0.063652,0.060875,0.063061,0.059587,0.053693,...,0.062520,0.063149,0.064969,0.066880,0.067833,0.067652,0.067921,0.066640,0.068619,0.071081
2,0.068674,0.062815,0.062630,0.060859,0.063652,0.060875,0.063061,0.059587,0.053693,0.053066,...,0.063149,0.064969,0.066880,0.067833,0.067652,0.067921,0.066640,0.068619,0.071081,0.070594
3,0.062815,0.062630,0.060859,0.063652,0.060875,0.063061,0.059587,0.053693,0.053066,0.053579,...,0.064969,0.066880,0.067833,0.067652,0.067921,0.066640,0.068619,0.071081,0.070594,0.069391
4,0.062630,0.060859,0.063652,0.060875,0.063061,0.059587,0.053693,0.053066,0.053579,0.047292,...,0.066880,0.067833,0.067652,0.067921,0.066640,0.068619,0.071081,0.070594,0.069391,0.070429


### D. Calculate the increase of the volume within a difference of 90 days in percent

In [10]:
entwicklungsrate_volumes_90 = (1/reframed_volumes['var1(t)']*reframed_volumes['var1(t+90)'])-1

##### Combine the developmentrate with the date

In [11]:
df_for_volumes['Entwicklungsrate Volume t+90'] = entwicklungsrate_volumes_90
df_for_volumes = df_for_volumes.drop(['Open','High','Low','Close','Volume','Mid_prices'], axis=1)
df_for_volumes.head()

,Date,Entwicklungsrate Volume t+90
0,2008-01-02,-0.090667
1,2008-01-03,-0.046851
2,2008-01-04,0.027965
3,2008-01-07,0.104684
4,2008-01-08,0.124521


###  E. Prepare dataframe for 5-day moving average

In [12]:
df_for_5_ma = df_ticker.copy(deep=True)
df_5_ma = df_for_5_ma.drop(['Date','Open','High','Low','Close','Volume'], axis=1)
#Normalize data
scaler = preprocessing.MinMaxScaler()
df_5_ma_values = scaler.fit_transform(df_midprices)
df_5_ma = pd.DataFrame(df_5_ma_values, columns=['Mid_prices'])
df_5_ma.head()

,Mid_prices
0,0.075460
1,0.074575
2,0.068674
3,0.062815
4,0.062630


#### Convert Series to Supervised Learning Dataframe

In [13]:
series_5_ma = df_5_ma.values
reframed_5_ma = series_to_supervised(series_5_ma, 0, 91)
reframed_5_ma.head()

,var1(t),var1(t+1),var1(t+2),var1(t+3),var1(t+4),var1(t+5),var1(t+6),var1(t+7),var1(t+8),var1(t+9),...,var1(t+81),var1(t+82),var1(t+83),var1(t+84),var1(t+85),var1(t+86),var1(t+87),var1(t+88),var1(t+89),var1(t+90)
0,0.075460,0.074575,0.068674,0.062815,0.062630,0.060859,0.063652,0.060875,0.063061,0.059587,...,0.060246,0.062520,0.063149,0.064969,0.066880,0.067833,0.067652,0.067921,0.066640,0.068619
1,0.074575,0.068674,0.062815,0.062630,0.060859,0.063652,0.060875,0.063061,0.059587,0.053693,...,0.062520,0.063149,0.064969,0.066880,0.067833,0.067652,0.067921,0.066640,0.068619,0.071081
2,0.068674,0.062815,0.062630,0.060859,0.063652,0.060875,0.063061,0.059587,0.053693,0.053066,...,0.063149,0.064969,0.066880,0.067833,0.067652,0.067921,0.066640,0.068619,0.071081,0.070594
3,0.062815,0.062630,0.060859,0.063652,0.060875,0.063061,0.059587,0.053693,0.053066,0.053579,...,0.064969,0.066880,0.067833,0.067652,0.067921,0.066640,0.068619,0.071081,0.070594,0.069391
4,0.062630,0.060859,0.063652,0.060875,0.063061,0.059587,0.053693,0.053066,0.053579,0.047292,...,0.066880,0.067833,0.067652,0.067921,0.066640,0.068619,0.071081,0.070594,0.069391,0.070429


In [14]:
entwicklungsrate_5_ma_0 = (reframed_5_ma['var1(t)']+reframed_5_ma['var1(t+1)']+reframed_5_ma['var1(t+2)']+reframed_5_ma['var1(t+3)']+reframed_5_ma['var1(t+4)'])/5
entwicklungsrate_5_ma_1 = (reframed_5_ma['var1(t+1)']+reframed_5_ma['var1(t+2)']+reframed_5_ma['var1(t+3)']+reframed_5_ma['var1(t+4)']+reframed_5_ma['var1(t+5)'])/5
entwicklungsrate_5_ma_2 = (reframed_5_ma['var1(t+2)']+reframed_5_ma['var1(t+3)']+reframed_5_ma['var1(t+4)']+reframed_5_ma['var1(t+5)']+reframed_5_ma['var1(t+6)'])/5
entwicklungsrate_5_ma_3 = (reframed_5_ma['var1(t+3)']+reframed_5_ma['var1(t+4)']+reframed_5_ma['var1(t+5)']+reframed_5_ma['var1(t+6)']+reframed_5_ma['var1(t+7)'])/5
entwicklungsrate_5_ma_4 = (reframed_5_ma['var1(t+4)']+reframed_5_ma['var1(t+5)']+reframed_5_ma['var1(t+6)']+reframed_5_ma['var1(t+7)']+reframed_5_ma['var1(t+8)'])/5
entwicklungsrate_5_ma_5 = (reframed_5_ma['var1(t+5)']+reframed_5_ma['var1(t+6)']+reframed_5_ma['var1(t+7)']+reframed_5_ma['var1(t+8)']+reframed_5_ma['var1(t+9)'])/5
entwicklungsrate_5_ma_6 = (reframed_5_ma['var1(t+6)']+reframed_5_ma['var1(t+7)']+reframed_5_ma['var1(t+8)']+reframed_5_ma['var1(t+9)']+reframed_5_ma['var1(t+10)'])/5
entwicklungsrate_5_ma_7 = (reframed_5_ma['var1(t+7)']+reframed_5_ma['var1(t+8)']+reframed_5_ma['var1(t+9)']+reframed_5_ma['var1(t+10)']+reframed_5_ma['var1(t+11)'])/5
entwicklungsrate_5_ma_8 = (reframed_5_ma['var1(t+8)']+reframed_5_ma['var1(t+9)']+reframed_5_ma['var1(t+10)']+reframed_5_ma['var1(t+11)']+reframed_5_ma['var1(t+12)'])/5
entwicklungsrate_5_ma_9 = (reframed_5_ma['var1(t+9)']+reframed_5_ma['var1(t+10)']+reframed_5_ma['var1(t+11)']+reframed_5_ma['var1(t+12)']+reframed_5_ma['var1(t+13)'])/5
entwicklungsrate_5_ma_10 = (reframed_5_ma['var1(t+10)']+reframed_5_ma['var1(t+11)']+reframed_5_ma['var1(t+12)']+reframed_5_ma['var1(t+13)']+reframed_5_ma['var1(t+14)'])/5
entwicklungsrate_5_ma_11 = (reframed_5_ma['var1(t+11)']+reframed_5_ma['var1(t+12)']+reframed_5_ma['var1(t+13)']+reframed_5_ma['var1(t+14)']+reframed_5_ma['var1(t+15)'])/5
entwicklungsrate_5_ma_12 = (reframed_5_ma['var1(t+12)']+reframed_5_ma['var1(t+13)']+reframed_5_ma['var1(t+14)']+reframed_5_ma['var1(t+15)']+reframed_5_ma['var1(t+16)'])/5
entwicklungsrate_5_ma_13 = (reframed_5_ma['var1(t+13)']+reframed_5_ma['var1(t+14)']+reframed_5_ma['var1(t+15)']+reframed_5_ma['var1(t+16)']+reframed_5_ma['var1(t+17)'])/5
entwicklungsrate_5_ma_14 = (reframed_5_ma['var1(t+14)']+reframed_5_ma['var1(t+15)']+reframed_5_ma['var1(t+16)']+reframed_5_ma['var1(t+17)']+reframed_5_ma['var1(t+18)'])/5
entwicklungsrate_5_ma_15 = (reframed_5_ma['var1(t+15)']+reframed_5_ma['var1(t+16)']+reframed_5_ma['var1(t+17)']+reframed_5_ma['var1(t+18)']+reframed_5_ma['var1(t+19)'])/5
entwicklungsrate_5_ma_16 = (reframed_5_ma['var1(t+16)']+reframed_5_ma['var1(t+17)']+reframed_5_ma['var1(t+18)']+reframed_5_ma['var1(t+19)']+reframed_5_ma['var1(t+20)'])/5
entwicklungsrate_5_ma_17 = (reframed_5_ma['var1(t+17)']+reframed_5_ma['var1(t+18)']+reframed_5_ma['var1(t+19)']+reframed_5_ma['var1(t+20)']+reframed_5_ma['var1(t+21)'])/5
entwicklungsrate_5_ma_18 = (reframed_5_ma['var1(t+18)']+reframed_5_ma['var1(t+19)']+reframed_5_ma['var1(t+20)']+reframed_5_ma['var1(t+21)']+reframed_5_ma['var1(t+22)'])/5
entwicklungsrate_5_ma_19 = (reframed_5_ma['var1(t+19)']+reframed_5_ma['var1(t+20)']+reframed_5_ma['var1(t+21)']+reframed_5_ma['var1(t+22)']+reframed_5_ma['var1(t+23)'])/5
entwicklungsrate_5_ma_20 = (reframed_5_ma['var1(t+20)']+reframed_5_ma['var1(t+21)']+reframed_5_ma['var1(t+22)']+reframed_5_ma['var1(t+23)']+reframed_5_ma['var1(t+24)'])/5
entwicklungsrate_5_ma_21 = (reframed_5_ma['var1(t+21)']+reframed_5_ma['var1(t+22)']+reframed_5_ma['var1(t+23)']+reframed_5_ma['var1(t+24)']+reframed_5_ma['var1(t+25)'])/5
entwicklungsrate_5_ma_22 = (reframed_5_ma['var1(t+22)']+reframed_5_ma['var1(t+23)']+reframed_5_ma['var1(t+24)']+reframed_5_ma['var1(t+25)']+reframed_5_ma['var1(t+26)'])/5
entwicklungsrate_5_ma_23 = (reframed_5_ma['var1(t+23)']+reframed_5_ma['var1(t+24)']+reframed_5_ma['var1(t+25)']+reframed_5_ma['var1(t+26)']+reframed_5_ma['var1(t+27)'])/5
entwicklungsrate_5_ma_24 = (reframed_5_ma['var1(t+24)']+reframed_5_ma['var1(t+25)']+reframed_5_ma['var1(t+26)']+reframed_5_ma['var1(t+27)']+reframed_5_ma['var1(t+28)'])/5
entwicklungsrate_5_ma_25 = (reframed_5_ma['var1(t+25)']+reframed_5_ma['var1(t+26)']+reframed_5_ma['var1(t+27)']+reframed_5_ma['var1(t+28)']+reframed_5_ma['var1(t+29)'])/5
entwicklungsrate_5_ma_26 = (reframed_5_ma['var1(t+26)']+reframed_5_ma['var1(t+27)']+reframed_5_ma['var1(t+28)']+reframed_5_ma['var1(t+29)']+reframed_5_ma['var1(t+30)'])/5
entwicklungsrate_5_ma_27 = (reframed_5_ma['var1(t+27)']+reframed_5_ma['var1(t+28)']+reframed_5_ma['var1(t+29)']+reframed_5_ma['var1(t+30)']+reframed_5_ma['var1(t+31)'])/5
entwicklungsrate_5_ma_28 = (reframed_5_ma['var1(t+28)']+reframed_5_ma['var1(t+29)']+reframed_5_ma['var1(t+30)']+reframed_5_ma['var1(t+31)']+reframed_5_ma['var1(t+32)'])/5
entwicklungsrate_5_ma_29 = (reframed_5_ma['var1(t+29)']+reframed_5_ma['var1(t+30)']+reframed_5_ma['var1(t+31)']+reframed_5_ma['var1(t+32)']+reframed_5_ma['var1(t+33)'])/5
entwicklungsrate_5_ma_30 = (reframed_5_ma['var1(t+30)']+reframed_5_ma['var1(t+31)']+reframed_5_ma['var1(t+32)']+reframed_5_ma['var1(t+33)']+reframed_5_ma['var1(t+34)'])/5
entwicklungsrate_5_ma_31 = (reframed_5_ma['var1(t+31)']+reframed_5_ma['var1(t+32)']+reframed_5_ma['var1(t+33)']+reframed_5_ma['var1(t+34)']+reframed_5_ma['var1(t+35)'])/5
entwicklungsrate_5_ma_32 = (reframed_5_ma['var1(t+32)']+reframed_5_ma['var1(t+33)']+reframed_5_ma['var1(t+34)']+reframed_5_ma['var1(t+35)']+reframed_5_ma['var1(t+36)'])/5
entwicklungsrate_5_ma_33 = (reframed_5_ma['var1(t+33)']+reframed_5_ma['var1(t+34)']+reframed_5_ma['var1(t+35)']+reframed_5_ma['var1(t+36)']+reframed_5_ma['var1(t+37)'])/5
entwicklungsrate_5_ma_34 = (reframed_5_ma['var1(t+34)']+reframed_5_ma['var1(t+35)']+reframed_5_ma['var1(t+36)']+reframed_5_ma['var1(t+37)']+reframed_5_ma['var1(t+38)'])/5
entwicklungsrate_5_ma_35 = (reframed_5_ma['var1(t+35)']+reframed_5_ma['var1(t+36)']+reframed_5_ma['var1(t+37)']+reframed_5_ma['var1(t+38)']+reframed_5_ma['var1(t+39)'])/5
entwicklungsrate_5_ma_36 = (reframed_5_ma['var1(t+36)']+reframed_5_ma['var1(t+37)']+reframed_5_ma['var1(t+38)']+reframed_5_ma['var1(t+39)']+reframed_5_ma['var1(t+40)'])/5
entwicklungsrate_5_ma_37 = (reframed_5_ma['var1(t+37)']+reframed_5_ma['var1(t+38)']+reframed_5_ma['var1(t+39)']+reframed_5_ma['var1(t+40)']+reframed_5_ma['var1(t+41)'])/5
entwicklungsrate_5_ma_38 = (reframed_5_ma['var1(t+38)']+reframed_5_ma['var1(t+39)']+reframed_5_ma['var1(t+40)']+reframed_5_ma['var1(t+41)']+reframed_5_ma['var1(t+42)'])/5
entwicklungsrate_5_ma_39 = (reframed_5_ma['var1(t+39)']+reframed_5_ma['var1(t+40)']+reframed_5_ma['var1(t+41)']+reframed_5_ma['var1(t+42)']+reframed_5_ma['var1(t+43)'])/5
entwicklungsrate_5_ma_40 = (reframed_5_ma['var1(t+40)']+reframed_5_ma['var1(t+41)']+reframed_5_ma['var1(t+42)']+reframed_5_ma['var1(t+43)']+reframed_5_ma['var1(t+44)'])/5
entwicklungsrate_5_ma_41 = (reframed_5_ma['var1(t+41)']+reframed_5_ma['var1(t+42)']+reframed_5_ma['var1(t+43)']+reframed_5_ma['var1(t+44)']+reframed_5_ma['var1(t+45)'])/5
entwicklungsrate_5_ma_42 = (reframed_5_ma['var1(t+42)']+reframed_5_ma['var1(t+43)']+reframed_5_ma['var1(t+44)']+reframed_5_ma['var1(t+45)']+reframed_5_ma['var1(t+46)'])/5
entwicklungsrate_5_ma_43 = (reframed_5_ma['var1(t+43)']+reframed_5_ma['var1(t+44)']+reframed_5_ma['var1(t+45)']+reframed_5_ma['var1(t+46)']+reframed_5_ma['var1(t+47)'])/5
entwicklungsrate_5_ma_44 = (reframed_5_ma['var1(t+44)']+reframed_5_ma['var1(t+45)']+reframed_5_ma['var1(t+46)']+reframed_5_ma['var1(t+47)']+reframed_5_ma['var1(t+48)'])/5
entwicklungsrate_5_ma_45 = (reframed_5_ma['var1(t+45)']+reframed_5_ma['var1(t+46)']+reframed_5_ma['var1(t+47)']+reframed_5_ma['var1(t+48)']+reframed_5_ma['var1(t+49)'])/5
entwicklungsrate_5_ma_46 = (reframed_5_ma['var1(t+46)']+reframed_5_ma['var1(t+47)']+reframed_5_ma['var1(t+48)']+reframed_5_ma['var1(t+49)']+reframed_5_ma['var1(t+50)'])/5
entwicklungsrate_5_ma_47 = (reframed_5_ma['var1(t+47)']+reframed_5_ma['var1(t+48)']+reframed_5_ma['var1(t+49)']+reframed_5_ma['var1(t+50)']+reframed_5_ma['var1(t+51)'])/5
entwicklungsrate_5_ma_48 = (reframed_5_ma['var1(t+48)']+reframed_5_ma['var1(t+49)']+reframed_5_ma['var1(t+50)']+reframed_5_ma['var1(t+51)']+reframed_5_ma['var1(t+52)'])/5
entwicklungsrate_5_ma_49 = (reframed_5_ma['var1(t+49)']+reframed_5_ma['var1(t+50)']+reframed_5_ma['var1(t+51)']+reframed_5_ma['var1(t+52)']+reframed_5_ma['var1(t+53)'])/5
entwicklungsrate_5_ma_50 = (reframed_5_ma['var1(t+50)']+reframed_5_ma['var1(t+51)']+reframed_5_ma['var1(t+52)']+reframed_5_ma['var1(t+53)']+reframed_5_ma['var1(t+54)'])/5
entwicklungsrate_5_ma_51 = (reframed_5_ma['var1(t+51)']+reframed_5_ma['var1(t+52)']+reframed_5_ma['var1(t+53)']+reframed_5_ma['var1(t+54)']+reframed_5_ma['var1(t+55)'])/5
entwicklungsrate_5_ma_52 = (reframed_5_ma['var1(t+52)']+reframed_5_ma['var1(t+53)']+reframed_5_ma['var1(t+54)']+reframed_5_ma['var1(t+55)']+reframed_5_ma['var1(t+56)'])/5
entwicklungsrate_5_ma_53 = (reframed_5_ma['var1(t+53)']+reframed_5_ma['var1(t+54)']+reframed_5_ma['var1(t+55)']+reframed_5_ma['var1(t+56)']+reframed_5_ma['var1(t+57)'])/5
entwicklungsrate_5_ma_54 = (reframed_5_ma['var1(t+54)']+reframed_5_ma['var1(t+55)']+reframed_5_ma['var1(t+56)']+reframed_5_ma['var1(t+57)']+reframed_5_ma['var1(t+58)'])/5
entwicklungsrate_5_ma_55 = (reframed_5_ma['var1(t+55)']+reframed_5_ma['var1(t+56)']+reframed_5_ma['var1(t+57)']+reframed_5_ma['var1(t+58)']+reframed_5_ma['var1(t+59)'])/5
entwicklungsrate_5_ma_56 = (reframed_5_ma['var1(t+56)']+reframed_5_ma['var1(t+57)']+reframed_5_ma['var1(t+58)']+reframed_5_ma['var1(t+59)']+reframed_5_ma['var1(t+60)'])/5
entwicklungsrate_5_ma_57 = (reframed_5_ma['var1(t+57)']+reframed_5_ma['var1(t+58)']+reframed_5_ma['var1(t+59)']+reframed_5_ma['var1(t+60)']+reframed_5_ma['var1(t+61)'])/5
entwicklungsrate_5_ma_58 = (reframed_5_ma['var1(t+58)']+reframed_5_ma['var1(t+59)']+reframed_5_ma['var1(t+60)']+reframed_5_ma['var1(t+61)']+reframed_5_ma['var1(t+62)'])/5
entwicklungsrate_5_ma_59 = (reframed_5_ma['var1(t+59)']+reframed_5_ma['var1(t+60)']+reframed_5_ma['var1(t+61)']+reframed_5_ma['var1(t+62)']+reframed_5_ma['var1(t+63)'])/5
entwicklungsrate_5_ma_60 = (reframed_5_ma['var1(t+60)']+reframed_5_ma['var1(t+61)']+reframed_5_ma['var1(t+62)']+reframed_5_ma['var1(t+63)']+reframed_5_ma['var1(t+64)'])/5
entwicklungsrate_5_ma_61 = (reframed_5_ma['var1(t+61)']+reframed_5_ma['var1(t+62)']+reframed_5_ma['var1(t+63)']+reframed_5_ma['var1(t+64)']+reframed_5_ma['var1(t+65)'])/5
entwicklungsrate_5_ma_62 = (reframed_5_ma['var1(t+62)']+reframed_5_ma['var1(t+63)']+reframed_5_ma['var1(t+64)']+reframed_5_ma['var1(t+65)']+reframed_5_ma['var1(t+66)'])/5
entwicklungsrate_5_ma_63 = (reframed_5_ma['var1(t+63)']+reframed_5_ma['var1(t+64)']+reframed_5_ma['var1(t+65)']+reframed_5_ma['var1(t+66)']+reframed_5_ma['var1(t+67)'])/5
entwicklungsrate_5_ma_64 = (reframed_5_ma['var1(t+64)']+reframed_5_ma['var1(t+65)']+reframed_5_ma['var1(t+66)']+reframed_5_ma['var1(t+67)']+reframed_5_ma['var1(t+68)'])/5
entwicklungsrate_5_ma_65 = (reframed_5_ma['var1(t+65)']+reframed_5_ma['var1(t+66)']+reframed_5_ma['var1(t+67)']+reframed_5_ma['var1(t+68)']+reframed_5_ma['var1(t+69)'])/5
entwicklungsrate_5_ma_66 = (reframed_5_ma['var1(t+66)']+reframed_5_ma['var1(t+67)']+reframed_5_ma['var1(t+68)']+reframed_5_ma['var1(t+69)']+reframed_5_ma['var1(t+70)'])/5
entwicklungsrate_5_ma_67 = (reframed_5_ma['var1(t+67)']+reframed_5_ma['var1(t+68)']+reframed_5_ma['var1(t+69)']+reframed_5_ma['var1(t+70)']+reframed_5_ma['var1(t+71)'])/5
entwicklungsrate_5_ma_68 = (reframed_5_ma['var1(t+68)']+reframed_5_ma['var1(t+69)']+reframed_5_ma['var1(t+70)']+reframed_5_ma['var1(t+71)']+reframed_5_ma['var1(t+72)'])/5
entwicklungsrate_5_ma_69 = (reframed_5_ma['var1(t+69)']+reframed_5_ma['var1(t+70)']+reframed_5_ma['var1(t+71)']+reframed_5_ma['var1(t+72)']+reframed_5_ma['var1(t+73)'])/5
entwicklungsrate_5_ma_70 = (reframed_5_ma['var1(t+70)']+reframed_5_ma['var1(t+71)']+reframed_5_ma['var1(t+72)']+reframed_5_ma['var1(t+73)']+reframed_5_ma['var1(t+74)'])/5
entwicklungsrate_5_ma_71 = (reframed_5_ma['var1(t+71)']+reframed_5_ma['var1(t+72)']+reframed_5_ma['var1(t+73)']+reframed_5_ma['var1(t+74)']+reframed_5_ma['var1(t+75)'])/5
entwicklungsrate_5_ma_72 = (reframed_5_ma['var1(t+72)']+reframed_5_ma['var1(t+73)']+reframed_5_ma['var1(t+74)']+reframed_5_ma['var1(t+75)']+reframed_5_ma['var1(t+76)'])/5
entwicklungsrate_5_ma_73 = (reframed_5_ma['var1(t+73)']+reframed_5_ma['var1(t+74)']+reframed_5_ma['var1(t+75)']+reframed_5_ma['var1(t+76)']+reframed_5_ma['var1(t+77)'])/5
entwicklungsrate_5_ma_74 = (reframed_5_ma['var1(t+74)']+reframed_5_ma['var1(t+75)']+reframed_5_ma['var1(t+76)']+reframed_5_ma['var1(t+77)']+reframed_5_ma['var1(t+78)'])/5
entwicklungsrate_5_ma_75 = (reframed_5_ma['var1(t+75)']+reframed_5_ma['var1(t+76)']+reframed_5_ma['var1(t+77)']+reframed_5_ma['var1(t+78)']+reframed_5_ma['var1(t+79)'])/5
entwicklungsrate_5_ma_76 = (reframed_5_ma['var1(t+76)']+reframed_5_ma['var1(t+77)']+reframed_5_ma['var1(t+78)']+reframed_5_ma['var1(t+79)']+reframed_5_ma['var1(t+80)'])/5
entwicklungsrate_5_ma_77 = (reframed_5_ma['var1(t+77)']+reframed_5_ma['var1(t+78)']+reframed_5_ma['var1(t+79)']+reframed_5_ma['var1(t+80)']+reframed_5_ma['var1(t+81)'])/5
entwicklungsrate_5_ma_78 = (reframed_5_ma['var1(t+78)']+reframed_5_ma['var1(t+79)']+reframed_5_ma['var1(t+80)']+reframed_5_ma['var1(t+81)']+reframed_5_ma['var1(t+82)'])/5
entwicklungsrate_5_ma_79 = (reframed_5_ma['var1(t+79)']+reframed_5_ma['var1(t+80)']+reframed_5_ma['var1(t+81)']+reframed_5_ma['var1(t+82)']+reframed_5_ma['var1(t+83)'])/5
entwicklungsrate_5_ma_80 = (reframed_5_ma['var1(t+80)']+reframed_5_ma['var1(t+81)']+reframed_5_ma['var1(t+82)']+reframed_5_ma['var1(t+83)']+reframed_5_ma['var1(t+84)'])/5
entwicklungsrate_5_ma_81 = (reframed_5_ma['var1(t+81)']+reframed_5_ma['var1(t+82)']+reframed_5_ma['var1(t+83)']+reframed_5_ma['var1(t+84)']+reframed_5_ma['var1(t+85)'])/5
entwicklungsrate_5_ma_82 = (reframed_5_ma['var1(t+82)']+reframed_5_ma['var1(t+83)']+reframed_5_ma['var1(t+84)']+reframed_5_ma['var1(t+85)']+reframed_5_ma['var1(t+86)'])/5
entwicklungsrate_5_ma_83 = (reframed_5_ma['var1(t+83)']+reframed_5_ma['var1(t+84)']+reframed_5_ma['var1(t+85)']+reframed_5_ma['var1(t+86)']+reframed_5_ma['var1(t+87)'])/5
entwicklungsrate_5_ma_84 = (reframed_5_ma['var1(t+84)']+reframed_5_ma['var1(t+85)']+reframed_5_ma['var1(t+86)']+reframed_5_ma['var1(t+87)']+reframed_5_ma['var1(t+88)'])/5
entwicklungsrate_5_ma_85 = (reframed_5_ma['var1(t+85)']+reframed_5_ma['var1(t+86)']+reframed_5_ma['var1(t+87)']+reframed_5_ma['var1(t+88)']+reframed_5_ma['var1(t+89)'])/5
entwicklungsrate_5_ma_86 = (reframed_5_ma['var1(t+86)']+reframed_5_ma['var1(t+87)']+reframed_5_ma['var1(t+88)']+reframed_5_ma['var1(t+89)']+reframed_5_ma['var1(t+90)'])/5

##### Combine the developmentrate with the dataframe

In [15]:
df_for_5_ma['Entwicklungsrate 5-Day MA t+4'] = entwicklungsrate_5_ma_0
df_for_5_ma['Entwicklungsrate 5-Day MA t+5'] = entwicklungsrate_5_ma_1
df_for_5_ma['Entwicklungsrate 5-Day MA t+6'] = entwicklungsrate_5_ma_2
df_for_5_ma['Entwicklungsrate 5-Day MA t+7'] = entwicklungsrate_5_ma_3
df_for_5_ma['Entwicklungsrate 5-Day MA t+8'] = entwicklungsrate_5_ma_4
df_for_5_ma['Entwicklungsrate 5-Day MA t+9'] = entwicklungsrate_5_ma_5
df_for_5_ma['Entwicklungsrate 5-Day MA t+10'] = entwicklungsrate_5_ma_6
df_for_5_ma['Entwicklungsrate 5-Day MA t+11'] = entwicklungsrate_5_ma_7
df_for_5_ma['Entwicklungsrate 5-Day MA t+12'] = entwicklungsrate_5_ma_8
df_for_5_ma['Entwicklungsrate 5-Day MA t+13'] = entwicklungsrate_5_ma_9
df_for_5_ma['Entwicklungsrate 5-Day MA t+14'] = entwicklungsrate_5_ma_10
df_for_5_ma['Entwicklungsrate 5-Day MA t+15'] = entwicklungsrate_5_ma_11
df_for_5_ma['Entwicklungsrate 5-Day MA t+16'] = entwicklungsrate_5_ma_12
df_for_5_ma['Entwicklungsrate 5-Day MA t+17'] = entwicklungsrate_5_ma_13
df_for_5_ma['Entwicklungsrate 5-Day MA t+18'] = entwicklungsrate_5_ma_14
df_for_5_ma['Entwicklungsrate 5-Day MA t+19'] = entwicklungsrate_5_ma_15
df_for_5_ma['Entwicklungsrate 5-Day MA t+20'] = entwicklungsrate_5_ma_16
df_for_5_ma['Entwicklungsrate 5-Day MA t+21'] = entwicklungsrate_5_ma_17
df_for_5_ma['Entwicklungsrate 5-Day MA t+22'] = entwicklungsrate_5_ma_18
df_for_5_ma['Entwicklungsrate 5-Day MA t+23'] = entwicklungsrate_5_ma_19
df_for_5_ma['Entwicklungsrate 5-Day MA t+24'] = entwicklungsrate_5_ma_20
df_for_5_ma['Entwicklungsrate 5-Day MA t+25'] = entwicklungsrate_5_ma_21
df_for_5_ma['Entwicklungsrate 5-Day MA t+26'] = entwicklungsrate_5_ma_22
df_for_5_ma['Entwicklungsrate 5-Day MA t+27'] = entwicklungsrate_5_ma_23
df_for_5_ma['Entwicklungsrate 5-Day MA t+28'] = entwicklungsrate_5_ma_24
df_for_5_ma['Entwicklungsrate 5-Day MA t+29'] = entwicklungsrate_5_ma_25
df_for_5_ma['Entwicklungsrate 5-Day MA t+30'] = entwicklungsrate_5_ma_26
df_for_5_ma['Entwicklungsrate 5-Day MA t+31'] = entwicklungsrate_5_ma_27
df_for_5_ma['Entwicklungsrate 5-Day MA t+32'] = entwicklungsrate_5_ma_28
df_for_5_ma['Entwicklungsrate 5-Day MA t+33'] = entwicklungsrate_5_ma_29
df_for_5_ma['Entwicklungsrate 5-Day MA t+34'] = entwicklungsrate_5_ma_30
df_for_5_ma['Entwicklungsrate 5-Day MA t+35'] = entwicklungsrate_5_ma_31
df_for_5_ma['Entwicklungsrate 5-Day MA t+36'] = entwicklungsrate_5_ma_32
df_for_5_ma['Entwicklungsrate 5-Day MA t+37'] = entwicklungsrate_5_ma_33
df_for_5_ma['Entwicklungsrate 5-Day MA t+38'] = entwicklungsrate_5_ma_34
df_for_5_ma['Entwicklungsrate 5-Day MA t+39'] = entwicklungsrate_5_ma_35
df_for_5_ma['Entwicklungsrate 5-Day MA t+40'] = entwicklungsrate_5_ma_36
df_for_5_ma['Entwicklungsrate 5-Day MA t+41'] = entwicklungsrate_5_ma_37
df_for_5_ma['Entwicklungsrate 5-Day MA t+42'] = entwicklungsrate_5_ma_38
df_for_5_ma['Entwicklungsrate 5-Day MA t+43'] = entwicklungsrate_5_ma_39
df_for_5_ma['Entwicklungsrate 5-Day MA t+44'] = entwicklungsrate_5_ma_40
df_for_5_ma['Entwicklungsrate 5-Day MA t+45'] = entwicklungsrate_5_ma_41
df_for_5_ma['Entwicklungsrate 5-Day MA t+46'] = entwicklungsrate_5_ma_42
df_for_5_ma['Entwicklungsrate 5-Day MA t+47'] = entwicklungsrate_5_ma_43
df_for_5_ma['Entwicklungsrate 5-Day MA t+48'] = entwicklungsrate_5_ma_44
df_for_5_ma['Entwicklungsrate 5-Day MA t+49'] = entwicklungsrate_5_ma_45
df_for_5_ma['Entwicklungsrate 5-Day MA t+50'] = entwicklungsrate_5_ma_46
df_for_5_ma['Entwicklungsrate 5-Day MA t+51'] = entwicklungsrate_5_ma_47
df_for_5_ma['Entwicklungsrate 5-Day MA t+52'] = entwicklungsrate_5_ma_48
df_for_5_ma['Entwicklungsrate 5-Day MA t+53'] = entwicklungsrate_5_ma_49
df_for_5_ma['Entwicklungsrate 5-Day MA t+54'] = entwicklungsrate_5_ma_50
df_for_5_ma['Entwicklungsrate 5-Day MA t+55'] = entwicklungsrate_5_ma_51
df_for_5_ma['Entwicklungsrate 5-Day MA t+56'] = entwicklungsrate_5_ma_52
df_for_5_ma['Entwicklungsrate 5-Day MA t+57'] = entwicklungsrate_5_ma_53
df_for_5_ma['Entwicklungsrate 5-Day MA t+58'] = entwicklungsrate_5_ma_54
df_for_5_ma['Entwicklungsrate 5-Day MA t+59'] = entwicklungsrate_5_ma_55
df_for_5_ma['Entwicklungsrate 5-Day MA t+60'] = entwicklungsrate_5_ma_56
df_for_5_ma['Entwicklungsrate 5-Day MA t+61'] = entwicklungsrate_5_ma_57
df_for_5_ma['Entwicklungsrate 5-Day MA t+62'] = entwicklungsrate_5_ma_58
df_for_5_ma['Entwicklungsrate 5-Day MA t+63'] = entwicklungsrate_5_ma_59
df_for_5_ma['Entwicklungsrate 5-Day MA t+64'] = entwicklungsrate_5_ma_60
df_for_5_ma['Entwicklungsrate 5-Day MA t+65'] = entwicklungsrate_5_ma_61
df_for_5_ma['Entwicklungsrate 5-Day MA t+66'] = entwicklungsrate_5_ma_62
df_for_5_ma['Entwicklungsrate 5-Day MA t+67'] = entwicklungsrate_5_ma_63
df_for_5_ma['Entwicklungsrate 5-Day MA t+68'] = entwicklungsrate_5_ma_64
df_for_5_ma['Entwicklungsrate 5-Day MA t+69'] = entwicklungsrate_5_ma_65
df_for_5_ma['Entwicklungsrate 5-Day MA t+70'] = entwicklungsrate_5_ma_66
df_for_5_ma['Entwicklungsrate 5-Day MA t+71'] = entwicklungsrate_5_ma_67
df_for_5_ma['Entwicklungsrate 5-Day MA t+72'] = entwicklungsrate_5_ma_68
df_for_5_ma['Entwicklungsrate 5-Day MA t+73'] = entwicklungsrate_5_ma_69
df_for_5_ma['Entwicklungsrate 5-Day MA t+74'] = entwicklungsrate_5_ma_70
df_for_5_ma['Entwicklungsrate 5-Day MA t+75'] = entwicklungsrate_5_ma_71
df_for_5_ma['Entwicklungsrate 5-Day MA t+76'] = entwicklungsrate_5_ma_72
df_for_5_ma['Entwicklungsrate 5-Day MA t+77'] = entwicklungsrate_5_ma_73
df_for_5_ma['Entwicklungsrate 5-Day MA t+78'] = entwicklungsrate_5_ma_74
df_for_5_ma['Entwicklungsrate 5-Day MA t+79'] = entwicklungsrate_5_ma_75
df_for_5_ma['Entwicklungsrate 5-Day MA t+80'] = entwicklungsrate_5_ma_76
df_for_5_ma['Entwicklungsrate 5-Day MA t+81'] = entwicklungsrate_5_ma_77
df_for_5_ma['Entwicklungsrate 5-Day MA t+82'] = entwicklungsrate_5_ma_78
df_for_5_ma['Entwicklungsrate 5-Day MA t+83'] = entwicklungsrate_5_ma_79
df_for_5_ma['Entwicklungsrate 5-Day MA t+84'] = entwicklungsrate_5_ma_80
df_for_5_ma['Entwicklungsrate 5-Day MA t+85'] = entwicklungsrate_5_ma_81
df_for_5_ma['Entwicklungsrate 5-Day MA t+86'] = entwicklungsrate_5_ma_82
df_for_5_ma['Entwicklungsrate 5-Day MA t+87'] = entwicklungsrate_5_ma_83
df_for_5_ma['Entwicklungsrate 5-Day MA t+88'] = entwicklungsrate_5_ma_84
df_for_5_ma['Entwicklungsrate 5-Day MA t+89'] = entwicklungsrate_5_ma_85
df_for_5_ma['Entwicklungsrate 5-Day MA t+90'] = entwicklungsrate_5_ma_86

df_for_5_ma = df_for_5_ma.drop(['Open','High','Low','Close','Volume','Mid_prices'], axis=1)
df_for_5_ma.head()

,Date,Entwicklungsrate 5-Day MA t+4,Entwicklungsrate 5-Day MA t+5,Entwicklungsrate 5-Day MA t+6,Entwicklungsrate 5-Day MA t+7,Entwicklungsrate 5-Day MA t+8,Entwicklungsrate 5-Day MA t+9,Entwicklungsrate 5-Day MA t+10,Entwicklungsrate 5-Day MA t+11,Entwicklungsrate 5-Day MA t+12,...,Entwicklungsrate 5-Day MA t+81,Entwicklungsrate 5-Day MA t+82,Entwicklungsrate 5-Day MA t+83,Entwicklungsrate 5-Day MA t+84,Entwicklungsrate 5-Day MA t+85,Entwicklungsrate 5-Day MA t+86,Entwicklungsrate 5-Day MA t+87,Entwicklungsrate 5-Day MA t+88,Entwicklungsrate 5-Day MA t+89,Entwicklungsrate 5-Day MA t+90
0,2008-01-02,0.068831,0.065910,0.063726,0.062166,0.062215,0.061607,0.060174,0.058056,0.056597,...,0.057122,0.058873,0.060540,0.062029,0.063553,0.065070,0.066097,0.067051,0.067385,0.067733
1,2008-01-03,0.065910,0.063726,0.062166,0.062215,0.061607,0.060174,0.058056,0.056597,0.053443,...,0.058873,0.060540,0.062029,0.063553,0.065070,0.066097,0.067051,0.067385,0.067733,0.068382
2,2008-01-04,0.063726,0.062166,0.062215,0.061607,0.060174,0.058056,0.056597,0.053443,0.048400,...,0.060540,0.062029,0.063553,0.065070,0.066097,0.067051,0.067385,0.067733,0.068382,0.068971
3,2008-01-07,0.062166,0.062215,0.061607,0.060174,0.058056,0.056597,0.053443,0.048400,0.044961,...,0.062029,0.063553,0.065070,0.066097,0.067051,0.067385,0.067733,0.068382,0.068971,0.069265
4,2008-01-08,0.062215,0.061607,0.060174,0.058056,0.056597,0.053443,0.048400,0.044961,0.041387,...,0.063553,0.065070,0.066097,0.067051,0.067385,0.067733,0.068382,0.068971,0.069265,0.070023


### F. Prepare dataframe for Exponential moving average

In [16]:
df_for_ema = df_ticker.copy(deep=True)
df_ema = df_for_ema.drop(['Date','Open','High','Low','Close','Volume'], axis=1)
#Normalize data
scaler = preprocessing.MinMaxScaler()
df_ema_values = scaler.fit_transform(df_midprices)
df_ema = pd.DataFrame(df_ema_values, columns=['Mid_prices'])
df_ema.head()

,Mid_prices
0,0.075460
1,0.074575
2,0.068674
3,0.062815
4,0.062630


#### Convert Series to Supervised Learning Dataframe

In [17]:
series_ema = df_ema.values
reframed_ema = series_to_supervised(series_ema, 0, 91)
reframed_ema.head()

,var1(t),var1(t+1),var1(t+2),var1(t+3),var1(t+4),var1(t+5),var1(t+6),var1(t+7),var1(t+8),var1(t+9),...,var1(t+81),var1(t+82),var1(t+83),var1(t+84),var1(t+85),var1(t+86),var1(t+87),var1(t+88),var1(t+89),var1(t+90)
0,0.075460,0.074575,0.068674,0.062815,0.062630,0.060859,0.063652,0.060875,0.063061,0.059587,...,0.060246,0.062520,0.063149,0.064969,0.066880,0.067833,0.067652,0.067921,0.066640,0.068619
1,0.074575,0.068674,0.062815,0.062630,0.060859,0.063652,0.060875,0.063061,0.059587,0.053693,...,0.062520,0.063149,0.064969,0.066880,0.067833,0.067652,0.067921,0.066640,0.068619,0.071081
2,0.068674,0.062815,0.062630,0.060859,0.063652,0.060875,0.063061,0.059587,0.053693,0.053066,...,0.063149,0.064969,0.066880,0.067833,0.067652,0.067921,0.066640,0.068619,0.071081,0.070594
3,0.062815,0.062630,0.060859,0.063652,0.060875,0.063061,0.059587,0.053693,0.053066,0.053579,...,0.064969,0.066880,0.067833,0.067652,0.067921,0.066640,0.068619,0.071081,0.070594,0.069391
4,0.062630,0.060859,0.063652,0.060875,0.063061,0.059587,0.053693,0.053066,0.053579,0.047292,...,0.066880,0.067833,0.067652,0.067921,0.066640,0.068619,0.071081,0.070594,0.069391,0.070429


In [18]:
entwicklungsrate_ema_0 = reframed_ema['var1(t)'] 
entwicklungsrate_ema_1 = (reframed_ema['var1(t+1)'] * (2/(90+1))) + (entwicklungsrate_ema_0 *  (1-(2/(90+1))))
entwicklungsrate_ema_2 = (reframed_ema['var1(t+2)'] * (2/(90+1))) + (entwicklungsrate_ema_1 *  (1-(2/(90+1))))
entwicklungsrate_ema_3 = (reframed_ema['var1(t+3)'] * (2/(90+1))) + (entwicklungsrate_ema_2 *  (1-(2/(90+1))))
entwicklungsrate_ema_4 = (reframed_ema['var1(t+4)'] * (2/(90+1))) + (entwicklungsrate_ema_3 *  (1-(2/(90+1))))
entwicklungsrate_ema_5 = (reframed_ema['var1(t+5)'] * (2/(90+1))) + (entwicklungsrate_ema_4 *  (1-(2/(90+1))))
entwicklungsrate_ema_6 = (reframed_ema['var1(t+6)'] * (2/(90+1))) + (entwicklungsrate_ema_5 *  (1-(2/(90+1))))
entwicklungsrate_ema_7 = (reframed_ema['var1(t+7)'] * (2/(90+1))) + (entwicklungsrate_ema_6 *  (1-(2/(90+1))))
entwicklungsrate_ema_8 = (reframed_ema['var1(t+8)'] * (2/(90+1))) + (entwicklungsrate_ema_7 *  (1-(2/(90+1))))
entwicklungsrate_ema_9 = (reframed_ema['var1(t+9)'] * (2/(90+1))) + (entwicklungsrate_ema_8 *  (1-(2/(90+1))))
entwicklungsrate_ema_10 = (reframed_ema['var1(t+10)'] * (2/(90+1))) + (entwicklungsrate_ema_9 *  (1-(2/(90+1))))
entwicklungsrate_ema_11 = (reframed_ema['var1(t+11)'] * (2/(90+1))) + (entwicklungsrate_ema_10 *  (1-(2/(90+1))))
entwicklungsrate_ema_12 = (reframed_ema['var1(t+12)'] * (2/(90+1))) + (entwicklungsrate_ema_11 *  (1-(2/(90+1))))
entwicklungsrate_ema_13 = (reframed_ema['var1(t+13)'] * (2/(90+1))) + (entwicklungsrate_ema_12 *  (1-(2/(90+1))))
entwicklungsrate_ema_14 = (reframed_ema['var1(t+14)'] * (2/(90+1))) + (entwicklungsrate_ema_13 *  (1-(2/(90+1))))
entwicklungsrate_ema_15 = (reframed_ema['var1(t+15)'] * (2/(90+1))) + (entwicklungsrate_ema_14 *  (1-(2/(90+1))))
entwicklungsrate_ema_16 = (reframed_ema['var1(t+16)'] * (2/(90+1))) + (entwicklungsrate_ema_15 *  (1-(2/(90+1))))
entwicklungsrate_ema_17 = (reframed_ema['var1(t+17)'] * (2/(90+1))) + (entwicklungsrate_ema_16 *  (1-(2/(90+1))))
entwicklungsrate_ema_18 = (reframed_ema['var1(t+18)'] * (2/(90+1))) + (entwicklungsrate_ema_17 *  (1-(2/(90+1))))
entwicklungsrate_ema_19 = (reframed_ema['var1(t+19)'] * (2/(90+1))) + (entwicklungsrate_ema_18 *  (1-(2/(90+1))))
entwicklungsrate_ema_20 = (reframed_ema['var1(t+20)'] * (2/(90+1))) + (entwicklungsrate_ema_19 *  (1-(2/(90+1))))
entwicklungsrate_ema_21 = (reframed_ema['var1(t+21)'] * (2/(90+1))) + (entwicklungsrate_ema_20 *  (1-(2/(90+1))))
entwicklungsrate_ema_22 = (reframed_ema['var1(t+22)'] * (2/(90+1))) + (entwicklungsrate_ema_21 *  (1-(2/(90+1))))
entwicklungsrate_ema_23 = (reframed_ema['var1(t+23)'] * (2/(90+1))) + (entwicklungsrate_ema_22 *  (1-(2/(90+1))))
entwicklungsrate_ema_24 = (reframed_ema['var1(t+24)'] * (2/(90+1))) + (entwicklungsrate_ema_23 *  (1-(2/(90+1))))
entwicklungsrate_ema_25 = (reframed_ema['var1(t+25)'] * (2/(90+1))) + (entwicklungsrate_ema_24 *  (1-(2/(90+1))))
entwicklungsrate_ema_26 = (reframed_ema['var1(t+26)'] * (2/(90+1))) + (entwicklungsrate_ema_25 *  (1-(2/(90+1))))
entwicklungsrate_ema_27 = (reframed_ema['var1(t+27)'] * (2/(90+1))) + (entwicklungsrate_ema_26 *  (1-(2/(90+1))))
entwicklungsrate_ema_28 = (reframed_ema['var1(t+28)'] * (2/(90+1))) + (entwicklungsrate_ema_27 *  (1-(2/(90+1))))
entwicklungsrate_ema_29 = (reframed_ema['var1(t+29)'] * (2/(90+1))) + (entwicklungsrate_ema_28 *  (1-(2/(90+1))))
entwicklungsrate_ema_30 = (reframed_ema['var1(t+30)'] * (2/(90+1))) + (entwicklungsrate_ema_29 *  (1-(2/(90+1))))
entwicklungsrate_ema_31 = (reframed_ema['var1(t+31)'] * (2/(90+1))) + (entwicklungsrate_ema_30 *  (1-(2/(90+1))))
entwicklungsrate_ema_32 = (reframed_ema['var1(t+32)'] * (2/(90+1))) + (entwicklungsrate_ema_31 *  (1-(2/(90+1))))
entwicklungsrate_ema_33 = (reframed_ema['var1(t+33)'] * (2/(90+1))) + (entwicklungsrate_ema_32 *  (1-(2/(90+1))))
entwicklungsrate_ema_34 = (reframed_ema['var1(t+34)'] * (2/(90+1))) + (entwicklungsrate_ema_33 *  (1-(2/(90+1))))
entwicklungsrate_ema_35 = (reframed_ema['var1(t+35)'] * (2/(90+1))) + (entwicklungsrate_ema_34 *  (1-(2/(90+1))))
entwicklungsrate_ema_36 = (reframed_ema['var1(t+36)'] * (2/(90+1))) + (entwicklungsrate_ema_35 *  (1-(2/(90+1))))
entwicklungsrate_ema_37 = (reframed_ema['var1(t+37)'] * (2/(90+1))) + (entwicklungsrate_ema_36 *  (1-(2/(90+1))))
entwicklungsrate_ema_38 = (reframed_ema['var1(t+38)'] * (2/(90+1))) + (entwicklungsrate_ema_37 *  (1-(2/(90+1))))
entwicklungsrate_ema_39 = (reframed_ema['var1(t+39)'] * (2/(90+1))) + (entwicklungsrate_ema_38 *  (1-(2/(90+1))))
entwicklungsrate_ema_40 = (reframed_ema['var1(t+40)'] * (2/(90+1))) + (entwicklungsrate_ema_39 *  (1-(2/(90+1))))
entwicklungsrate_ema_41 = (reframed_ema['var1(t+41)'] * (2/(90+1))) + (entwicklungsrate_ema_40 *  (1-(2/(90+1))))
entwicklungsrate_ema_42 = (reframed_ema['var1(t+42)'] * (2/(90+1))) + (entwicklungsrate_ema_41 *  (1-(2/(90+1))))
entwicklungsrate_ema_43 = (reframed_ema['var1(t+43)'] * (2/(90+1))) + (entwicklungsrate_ema_42 *  (1-(2/(90+1))))
entwicklungsrate_ema_44 = (reframed_ema['var1(t+44)'] * (2/(90+1))) + (entwicklungsrate_ema_43 *  (1-(2/(90+1))))
entwicklungsrate_ema_45 = (reframed_ema['var1(t+45)'] * (2/(90+1))) + (entwicklungsrate_ema_44 *  (1-(2/(90+1))))
entwicklungsrate_ema_46 = (reframed_ema['var1(t+46)'] * (2/(90+1))) + (entwicklungsrate_ema_45 *  (1-(2/(90+1))))
entwicklungsrate_ema_47 = (reframed_ema['var1(t+47)'] * (2/(90+1))) + (entwicklungsrate_ema_46 *  (1-(2/(90+1))))
entwicklungsrate_ema_48 = (reframed_ema['var1(t+48)'] * (2/(90+1))) + (entwicklungsrate_ema_47 *  (1-(2/(90+1))))
entwicklungsrate_ema_49 = (reframed_ema['var1(t+49)'] * (2/(90+1))) + (entwicklungsrate_ema_48 *  (1-(2/(90+1))))
entwicklungsrate_ema_50 = (reframed_ema['var1(t+50)'] * (2/(90+1))) + (entwicklungsrate_ema_49 *  (1-(2/(90+1))))
entwicklungsrate_ema_51 = (reframed_ema['var1(t+51)'] * (2/(90+1))) + (entwicklungsrate_ema_50 *  (1-(2/(90+1))))
entwicklungsrate_ema_52 = (reframed_ema['var1(t+52)'] * (2/(90+1))) + (entwicklungsrate_ema_51 *  (1-(2/(90+1))))
entwicklungsrate_ema_53 = (reframed_ema['var1(t+53)'] * (2/(90+1))) + (entwicklungsrate_ema_52 *  (1-(2/(90+1))))
entwicklungsrate_ema_54 = (reframed_ema['var1(t+54)'] * (2/(90+1))) + (entwicklungsrate_ema_53 *  (1-(2/(90+1))))
entwicklungsrate_ema_55 = (reframed_ema['var1(t+55)'] * (2/(90+1))) + (entwicklungsrate_ema_54 *  (1-(2/(90+1))))
entwicklungsrate_ema_56 = (reframed_ema['var1(t+56)'] * (2/(90+1))) + (entwicklungsrate_ema_55 *  (1-(2/(90+1))))
entwicklungsrate_ema_57 = (reframed_ema['var1(t+57)'] * (2/(90+1))) + (entwicklungsrate_ema_56 *  (1-(2/(90+1))))
entwicklungsrate_ema_58 = (reframed_ema['var1(t+58)'] * (2/(90+1))) + (entwicklungsrate_ema_57 *  (1-(2/(90+1))))
entwicklungsrate_ema_59 = (reframed_ema['var1(t+59)'] * (2/(90+1))) + (entwicklungsrate_ema_58 *  (1-(2/(90+1))))
entwicklungsrate_ema_60 = (reframed_ema['var1(t+60)'] * (2/(90+1))) + (entwicklungsrate_ema_59 *  (1-(2/(90+1))))
entwicklungsrate_ema_61 = (reframed_ema['var1(t+61)'] * (2/(90+1))) + (entwicklungsrate_ema_60 *  (1-(2/(90+1))))
entwicklungsrate_ema_62 = (reframed_ema['var1(t+62)'] * (2/(90+1))) + (entwicklungsrate_ema_61 *  (1-(2/(90+1))))
entwicklungsrate_ema_63 = (reframed_ema['var1(t+63)'] * (2/(90+1))) + (entwicklungsrate_ema_62 *  (1-(2/(90+1))))
entwicklungsrate_ema_64 = (reframed_ema['var1(t+64)'] * (2/(90+1))) + (entwicklungsrate_ema_63 *  (1-(2/(90+1))))
entwicklungsrate_ema_65 = (reframed_ema['var1(t+65)'] * (2/(90+1))) + (entwicklungsrate_ema_64 *  (1-(2/(90+1))))
entwicklungsrate_ema_66 = (reframed_ema['var1(t+66)'] * (2/(90+1))) + (entwicklungsrate_ema_65 *  (1-(2/(90+1))))
entwicklungsrate_ema_67 = (reframed_ema['var1(t+67)'] * (2/(90+1))) + (entwicklungsrate_ema_66 *  (1-(2/(90+1))))
entwicklungsrate_ema_68 = (reframed_ema['var1(t+68)'] * (2/(90+1))) + (entwicklungsrate_ema_67 *  (1-(2/(90+1))))
entwicklungsrate_ema_69 = (reframed_ema['var1(t+69)'] * (2/(90+1))) + (entwicklungsrate_ema_68 *  (1-(2/(90+1))))
entwicklungsrate_ema_70 = (reframed_ema['var1(t+70)'] * (2/(90+1))) + (entwicklungsrate_ema_69 *  (1-(2/(90+1))))
entwicklungsrate_ema_71 = (reframed_ema['var1(t+71)'] * (2/(90+1))) + (entwicklungsrate_ema_70 *  (1-(2/(90+1))))
entwicklungsrate_ema_72 = (reframed_ema['var1(t+72)'] * (2/(90+1))) + (entwicklungsrate_ema_71 *  (1-(2/(90+1))))
entwicklungsrate_ema_73 = (reframed_ema['var1(t+73)'] * (2/(90+1))) + (entwicklungsrate_ema_72 *  (1-(2/(90+1))))
entwicklungsrate_ema_74 = (reframed_ema['var1(t+74)'] * (2/(90+1))) + (entwicklungsrate_ema_73 *  (1-(2/(90+1))))
entwicklungsrate_ema_75 = (reframed_ema['var1(t+75)'] * (2/(90+1))) + (entwicklungsrate_ema_74 *  (1-(2/(90+1))))
entwicklungsrate_ema_76 = (reframed_ema['var1(t+76)'] * (2/(90+1))) + (entwicklungsrate_ema_75 *  (1-(2/(90+1))))
entwicklungsrate_ema_77 = (reframed_ema['var1(t+77)'] * (2/(90+1))) + (entwicklungsrate_ema_76 *  (1-(2/(90+1))))
entwicklungsrate_ema_78 = (reframed_ema['var1(t+78)'] * (2/(90+1))) + (entwicklungsrate_ema_77 *  (1-(2/(90+1))))
entwicklungsrate_ema_79 = (reframed_ema['var1(t+79)'] * (2/(90+1))) + (entwicklungsrate_ema_78 *  (1-(2/(90+1))))
entwicklungsrate_ema_80 = (reframed_ema['var1(t+80)'] * (2/(90+1))) + (entwicklungsrate_ema_79 *  (1-(2/(90+1))))
entwicklungsrate_ema_81 = (reframed_ema['var1(t+81)'] * (2/(90+1))) + (entwicklungsrate_ema_80 *  (1-(2/(90+1))))
entwicklungsrate_ema_82 = (reframed_ema['var1(t+82)'] * (2/(90+1))) + (entwicklungsrate_ema_81 *  (1-(2/(90+1))))
entwicklungsrate_ema_83 = (reframed_ema['var1(t+83)'] * (2/(90+1))) + (entwicklungsrate_ema_82 *  (1-(2/(90+1))))
entwicklungsrate_ema_84 = (reframed_ema['var1(t+84)'] * (2/(90+1))) + (entwicklungsrate_ema_83 *  (1-(2/(90+1))))
entwicklungsrate_ema_85 = (reframed_ema['var1(t+85)'] * (2/(90+1))) + (entwicklungsrate_ema_84 *  (1-(2/(90+1))))
entwicklungsrate_ema_86 = (reframed_ema['var1(t+86)'] * (2/(90+1))) + (entwicklungsrate_ema_85 *  (1-(2/(90+1))))
entwicklungsrate_ema_87 = (reframed_ema['var1(t+87)'] * (2/(90+1))) + (entwicklungsrate_ema_86 *  (1-(2/(90+1))))
entwicklungsrate_ema_88 = (reframed_ema['var1(t+88)'] * (2/(90+1))) + (entwicklungsrate_ema_87 *  (1-(2/(90+1))))
entwicklungsrate_ema_89 = (reframed_ema['var1(t+89)'] * (2/(90+1))) + (entwicklungsrate_ema_88 *  (1-(2/(90+1))))
entwicklungsrate_ema_90 = (reframed_ema['var1(t+90)'] * (2/(90+1))) + (entwicklungsrate_ema_89 *  (1-(2/(90+1))))


In [19]:
df_for_ema['Entwicklungsrate EMA t+0'] = entwicklungsrate_ema_0
df_for_ema['Entwicklungsrate EMA t+1'] = entwicklungsrate_ema_1
df_for_ema['Entwicklungsrate EMA t+2'] = entwicklungsrate_ema_2
df_for_ema['Entwicklungsrate EMA t+3'] = entwicklungsrate_ema_3
df_for_ema['Entwicklungsrate EMA t+4'] = entwicklungsrate_ema_4
df_for_ema['Entwicklungsrate EMA t+5'] = entwicklungsrate_ema_5
df_for_ema['Entwicklungsrate EMA t+6'] = entwicklungsrate_ema_6
df_for_ema['Entwicklungsrate EMA t+7'] = entwicklungsrate_ema_7
df_for_ema['Entwicklungsrate EMA t+8'] = entwicklungsrate_ema_8
df_for_ema['Entwicklungsrate EMA t+9'] = entwicklungsrate_ema_9
df_for_ema['Entwicklungsrate EMA t+10'] = entwicklungsrate_ema_10
df_for_ema['Entwicklungsrate EMA t+11'] = entwicklungsrate_ema_11
df_for_ema['Entwicklungsrate EMA t+12'] = entwicklungsrate_ema_12
df_for_ema['Entwicklungsrate EMA t+13'] = entwicklungsrate_ema_13
df_for_ema['Entwicklungsrate EMA t+14'] = entwicklungsrate_ema_14
df_for_ema['Entwicklungsrate EMA t+15'] = entwicklungsrate_ema_15
df_for_ema['Entwicklungsrate EMA t+16'] = entwicklungsrate_ema_16
df_for_ema['Entwicklungsrate EMA t+17'] = entwicklungsrate_ema_17
df_for_ema['Entwicklungsrate EMA t+18'] = entwicklungsrate_ema_18
df_for_ema['Entwicklungsrate EMA t+19'] = entwicklungsrate_ema_19
df_for_ema['Entwicklungsrate EMA t+20'] = entwicklungsrate_ema_20
df_for_ema['Entwicklungsrate EMA t+21'] = entwicklungsrate_ema_21
df_for_ema['Entwicklungsrate EMA t+22'] = entwicklungsrate_ema_22
df_for_ema['Entwicklungsrate EMA t+23'] = entwicklungsrate_ema_23
df_for_ema['Entwicklungsrate EMA t+24'] = entwicklungsrate_ema_24
df_for_ema['Entwicklungsrate EMA t+25'] = entwicklungsrate_ema_25
df_for_ema['Entwicklungsrate EMA t+26'] = entwicklungsrate_ema_26
df_for_ema['Entwicklungsrate EMA t+27'] = entwicklungsrate_ema_27
df_for_ema['Entwicklungsrate EMA t+28'] = entwicklungsrate_ema_28
df_for_ema['Entwicklungsrate EMA t+29'] = entwicklungsrate_ema_29
df_for_ema['Entwicklungsrate EMA t+30'] = entwicklungsrate_ema_30
df_for_ema['Entwicklungsrate EMA t+31'] = entwicklungsrate_ema_31
df_for_ema['Entwicklungsrate EMA t+32'] = entwicklungsrate_ema_32
df_for_ema['Entwicklungsrate EMA t+33'] = entwicklungsrate_ema_33
df_for_ema['Entwicklungsrate EMA t+34'] = entwicklungsrate_ema_34
df_for_ema['Entwicklungsrate EMA t+35'] = entwicklungsrate_ema_35
df_for_ema['Entwicklungsrate EMA t+36'] = entwicklungsrate_ema_36
df_for_ema['Entwicklungsrate EMA t+37'] = entwicklungsrate_ema_37
df_for_ema['Entwicklungsrate EMA t+38'] = entwicklungsrate_ema_38
df_for_ema['Entwicklungsrate EMA t+39'] = entwicklungsrate_ema_39
df_for_ema['Entwicklungsrate EMA t+40'] = entwicklungsrate_ema_40
df_for_ema['Entwicklungsrate EMA t+41'] = entwicklungsrate_ema_41
df_for_ema['Entwicklungsrate EMA t+42'] = entwicklungsrate_ema_42
df_for_ema['Entwicklungsrate EMA t+43'] = entwicklungsrate_ema_43
df_for_ema['Entwicklungsrate EMA t+44'] = entwicklungsrate_ema_44
df_for_ema['Entwicklungsrate EMA t+45'] = entwicklungsrate_ema_45
df_for_ema['Entwicklungsrate EMA t+46'] = entwicklungsrate_ema_46
df_for_ema['Entwicklungsrate EMA t+47'] = entwicklungsrate_ema_47
df_for_ema['Entwicklungsrate EMA t+48'] = entwicklungsrate_ema_48
df_for_ema['Entwicklungsrate EMA t+49'] = entwicklungsrate_ema_49
df_for_ema['Entwicklungsrate EMA t+50'] = entwicklungsrate_ema_50
df_for_ema['Entwicklungsrate EMA t+51'] = entwicklungsrate_ema_51
df_for_ema['Entwicklungsrate EMA t+52'] = entwicklungsrate_ema_52
df_for_ema['Entwicklungsrate EMA t+53'] = entwicklungsrate_ema_53
df_for_ema['Entwicklungsrate EMA t+54'] = entwicklungsrate_ema_54
df_for_ema['Entwicklungsrate EMA t+55'] = entwicklungsrate_ema_55
df_for_ema['Entwicklungsrate EMA t+56'] = entwicklungsrate_ema_56
df_for_ema['Entwicklungsrate EMA t+57'] = entwicklungsrate_ema_57
df_for_ema['Entwicklungsrate EMA t+58'] = entwicklungsrate_ema_58
df_for_ema['Entwicklungsrate EMA t+59'] = entwicklungsrate_ema_59
df_for_ema['Entwicklungsrate EMA t+60'] = entwicklungsrate_ema_60
df_for_ema['Entwicklungsrate EMA t+61'] = entwicklungsrate_ema_61
df_for_ema['Entwicklungsrate EMA t+62'] = entwicklungsrate_ema_62
df_for_ema['Entwicklungsrate EMA t+63'] = entwicklungsrate_ema_63
df_for_ema['Entwicklungsrate EMA t+64'] = entwicklungsrate_ema_64
df_for_ema['Entwicklungsrate EMA t+65'] = entwicklungsrate_ema_65
df_for_ema['Entwicklungsrate EMA t+66'] = entwicklungsrate_ema_66
df_for_ema['Entwicklungsrate EMA t+67'] = entwicklungsrate_ema_67
df_for_ema['Entwicklungsrate EMA t+68'] = entwicklungsrate_ema_68
df_for_ema['Entwicklungsrate EMA t+69'] = entwicklungsrate_ema_69
df_for_ema['Entwicklungsrate EMA t+70'] = entwicklungsrate_ema_70
df_for_ema['Entwicklungsrate EMA t+71'] = entwicklungsrate_ema_71
df_for_ema['Entwicklungsrate EMA t+72'] = entwicklungsrate_ema_72
df_for_ema['Entwicklungsrate EMA t+73'] = entwicklungsrate_ema_73
df_for_ema['Entwicklungsrate EMA t+74'] = entwicklungsrate_ema_74
df_for_ema['Entwicklungsrate EMA t+75'] = entwicklungsrate_ema_75
df_for_ema['Entwicklungsrate EMA t+76'] = entwicklungsrate_ema_76
df_for_ema['Entwicklungsrate EMA t+77'] = entwicklungsrate_ema_77
df_for_ema['Entwicklungsrate EMA t+78'] = entwicklungsrate_ema_78
df_for_ema['Entwicklungsrate EMA t+79'] = entwicklungsrate_ema_79
df_for_ema['Entwicklungsrate EMA t+80'] = entwicklungsrate_ema_80
df_for_ema['Entwicklungsrate EMA t+81'] = entwicklungsrate_ema_81
df_for_ema['Entwicklungsrate EMA t+82'] = entwicklungsrate_ema_82
df_for_ema['Entwicklungsrate EMA t+83'] = entwicklungsrate_ema_83
df_for_ema['Entwicklungsrate EMA t+84'] = entwicklungsrate_ema_84
df_for_ema['Entwicklungsrate EMA t+85'] = entwicklungsrate_ema_85
df_for_ema['Entwicklungsrate EMA t+86'] = entwicklungsrate_ema_86
df_for_ema['Entwicklungsrate EMA t+87'] = entwicklungsrate_ema_87
df_for_ema['Entwicklungsrate EMA t+88'] = entwicklungsrate_ema_88
df_for_ema['Entwicklungsrate EMA t+89'] = entwicklungsrate_ema_89
df_for_ema['Entwicklungsrate EMA t+90'] = entwicklungsrate_ema_90
df_for_ema = df_for_ema.drop(['Open','High','Low','Close','Volume','Mid_prices'], axis=1)
df_for_ema.head()

,Date,Entwicklungsrate EMA t+0,Entwicklungsrate EMA t+1,Entwicklungsrate EMA t+2,Entwicklungsrate EMA t+3,Entwicklungsrate EMA t+4,Entwicklungsrate EMA t+5,Entwicklungsrate EMA t+6,Entwicklungsrate EMA t+7,Entwicklungsrate EMA t+8,...,Entwicklungsrate EMA t+81,Entwicklungsrate EMA t+82,Entwicklungsrate EMA t+83,Entwicklungsrate EMA t+84,Entwicklungsrate EMA t+85,Entwicklungsrate EMA t+86,Entwicklungsrate EMA t+87,Entwicklungsrate EMA t+88,Entwicklungsrate EMA t+89,Entwicklungsrate EMA t+90
0,2008-01-02,0.075460,0.075441,0.075292,0.075018,0.074746,0.074440,0.074203,0.073910,0.073672,...,0.047610,0.047937,0.048272,0.048639,0.049039,0.049452,0.049852,0.050250,0.050610,0.051006
1,2008-01-03,0.074575,0.074445,0.074189,0.073935,0.073648,0.073428,0.073152,0.072931,0.072637,...,0.047794,0.048132,0.048502,0.048906,0.049322,0.049724,0.050124,0.050487,0.050886,0.051330
2,2008-01-04,0.068674,0.068545,0.068415,0.068249,0.068148,0.067988,0.067880,0.067697,0.067390,...,0.047178,0.047569,0.047993,0.048429,0.048852,0.049271,0.049652,0.050069,0.050531,0.050972
3,2008-01-07,0.062815,0.062811,0.062768,0.062787,0.062745,0.062752,0.062683,0.062485,0.062278,...,0.046622,0.047067,0.047523,0.047966,0.048404,0.048805,0.049240,0.049720,0.050179,0.050601
4,2008-01-08,0.062630,0.062591,0.062614,0.062576,0.062587,0.062521,0.062327,0.062123,0.061936,...,0.047037,0.047494,0.047937,0.048376,0.048778,0.049214,0.049694,0.050154,0.050576,0.051013


# 4. Merge all dataframes

In [20]:
merged_df = pd.merge(df_for_volumes,df_for_midprices,on='Date')
merged_df = pd.merge(merged_df,df_for_5_ma,on='Date')
merged_df = pd.merge(merged_df,df_for_ema,on='Date')
merged_df.head()

,Date,Entwicklungsrate Volume t+90,Entwicklungsrate Preis t+90,Entwicklungsrate 5-Day MA t+4,Entwicklungsrate 5-Day MA t+5,Entwicklungsrate 5-Day MA t+6,Entwicklungsrate 5-Day MA t+7,Entwicklungsrate 5-Day MA t+8,Entwicklungsrate 5-Day MA t+9,Entwicklungsrate 5-Day MA t+10,...,Entwicklungsrate EMA t+81,Entwicklungsrate EMA t+82,Entwicklungsrate EMA t+83,Entwicklungsrate EMA t+84,Entwicklungsrate EMA t+85,Entwicklungsrate EMA t+86,Entwicklungsrate EMA t+87,Entwicklungsrate EMA t+88,Entwicklungsrate EMA t+89,Entwicklungsrate EMA t+90
0,2008-01-02,-0.090667,-0.090667,0.068831,0.065910,0.063726,0.062166,0.062215,0.061607,0.060174,...,0.047610,0.047937,0.048272,0.048639,0.049039,0.049452,0.049852,0.050250,0.050610,0.051006
1,2008-01-03,-0.046851,-0.046851,0.065910,0.063726,0.062166,0.062215,0.061607,0.060174,0.058056,...,0.047794,0.048132,0.048502,0.048906,0.049322,0.049724,0.050124,0.050487,0.050886,0.051330
2,2008-01-04,0.027965,0.027965,0.063726,0.062166,0.062215,0.061607,0.060174,0.058056,0.056597,...,0.047178,0.047569,0.047993,0.048429,0.048852,0.049271,0.049652,0.050069,0.050531,0.050972
3,2008-01-07,0.104684,0.104684,0.062166,0.062215,0.061607,0.060174,0.058056,0.056597,0.053443,...,0.046622,0.047067,0.047523,0.047966,0.048404,0.048805,0.049240,0.049720,0.050179,0.050601
4,2008-01-08,0.124521,0.124521,0.062215,0.061607,0.060174,0.058056,0.056597,0.053443,0.048400,...,0.047037,0.047494,0.047937,0.048376,0.048778,0.049214,0.049694,0.050154,0.050576,0.051013


# 5. Add trained label

##### Load Trained Label csv file

In [21]:
df_train_label = pd.read_csv('data/labels_train.csv', header=0, index_col=0)
df_train_label.head()

,A,AAN,AAP,AAPL,AAXN,ABC,ABCD,ABG,ABM,ABMC,...,XEL,XOM,XPO,XRAY,XRX,XSPY,XYL,Y,YELP,ZBH
Date,,,,,,,,,,,,,,,,,,,,,
2008-01-02,0,1,0,-1,-1,0,0,0,0,-1,...,0,0,0,0,0,0,0,0,0,1
2008-01-03,0,1,0,-1,-1,0,0,0,0,-1,...,0,0,0,0,0,0,0,0,0,1
2008-01-04,0,1,0,0,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2008-01-07,0,1,0,0,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2008-01-08,0,1,0,0,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### Drop all columns except the one for 'AAPL'

In [22]:
df_train_label = df_train_label.loc[:, df_train_label.columns.intersection(['AAPL'])]
df_train_label.columns = ['Y']
df_train_label.head()

,Y
Date,
2008-01-02,-1
2008-01-03,-1
2008-01-04,0
2008-01-07,0
2008-01-08,0


##### Add the trained label dataframe to the others

In [23]:
df_complete = pd.merge(merged_df,df_train_label[['Y']],on='Date')
df_complete = df_complete.sort_values('Date')
df_complete = df_complete.drop('Date', axis=1)
df_complete.head()

,Entwicklungsrate Volume t+90,Entwicklungsrate Preis t+90,Entwicklungsrate 5-Day MA t+4,Entwicklungsrate 5-Day MA t+5,Entwicklungsrate 5-Day MA t+6,Entwicklungsrate 5-Day MA t+7,Entwicklungsrate 5-Day MA t+8,Entwicklungsrate 5-Day MA t+9,Entwicklungsrate 5-Day MA t+10,Entwicklungsrate 5-Day MA t+11,...,Entwicklungsrate EMA t+82,Entwicklungsrate EMA t+83,Entwicklungsrate EMA t+84,Entwicklungsrate EMA t+85,Entwicklungsrate EMA t+86,Entwicklungsrate EMA t+87,Entwicklungsrate EMA t+88,Entwicklungsrate EMA t+89,Entwicklungsrate EMA t+90,Y
0,-0.090667,-0.090667,0.068831,0.065910,0.063726,0.062166,0.062215,0.061607,0.060174,0.058056,...,0.047937,0.048272,0.048639,0.049039,0.049452,0.049852,0.050250,0.050610,0.051006,-1
1,-0.046851,-0.046851,0.065910,0.063726,0.062166,0.062215,0.061607,0.060174,0.058056,0.056597,...,0.048132,0.048502,0.048906,0.049322,0.049724,0.050124,0.050487,0.050886,0.051330,-1
2,0.027965,0.027965,0.063726,0.062166,0.062215,0.061607,0.060174,0.058056,0.056597,0.053443,...,0.047569,0.047993,0.048429,0.048852,0.049271,0.049652,0.050069,0.050531,0.050972,0
3,0.104684,0.104684,0.062166,0.062215,0.061607,0.060174,0.058056,0.056597,0.053443,0.048400,...,0.047067,0.047523,0.047966,0.048404,0.048805,0.049240,0.049720,0.050179,0.050601,0
4,0.124521,0.124521,0.062215,0.061607,0.060174,0.058056,0.056597,0.053443,0.048400,0.044961,...,0.047494,0.047937,0.048376,0.048778,0.049214,0.049694,0.050154,0.050576,0.051013,0


# 6. Save dataframe as .csv file

In [24]:
df_complete.to_csv('Prepared data/Training_set_AAPL.csv', encoding='utf-8')